<a href="https://colab.research.google.com/github/Xaomomo/Unet-with-attention/blob/main/Resnext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Descarga datos
Los datos se encuentran en el drive, por lo que usara gdown para sacarlos directamente y no tener que hacer la coneccion, ya que estamos descargando un zip.

In [ ]:
!gdown "1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1KJPU1D0ewHmU7Jzl1irq3Gb-JkUqV1VR&confirm=t
To: /content/input.zip
100% 367M/367M [00:09<00:00, 38.0MB/s]


In [ ]:
!unzip -q input.zip

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 1

In [ ]:
import nibabel as nib
import os
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
import torch

In [ ]:
import imageio as iio
import glob
from skimage.transform import resize
import PIL
from PIL import Image
src="/content/input/train"
imag="/images/"
X=np.zeros((len(glob.glob(src+imag+"*.png")),IMG_WIDTH,IMG_HEIGHT,1))
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x!="/content/input/train/images/coronacases_009_149.png":
    im=Image.open(x)
    X[i,:,:,0]=im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST)
mas="/Amasks/"
Y=np.zeros((len(glob.glob(src+mas+"*.png")),IMG_WIDTH,IMG_HEIGHT))
for i,x in enumerate(sorted(glob.glob(src+mas+"*.png"))):
  if x!="/content/input/train/Amasks/coronacases_009_149.png":
    im=Image.open(x)
    im=np.array(im.resize((IMG_WIDTH,IMG_HEIGHT),resample=PIL.Image.NEAREST))
    im[im==85]=1
    im[im==170]=2
    im[im==255]=3
    Y[i,:,:]=im
import tensorflow as tf
print(Y.shape)
print(np.unique(Y))
Y=tf.keras.utils.to_categorical(Y)
print(Y.shape)
print(np.unique(Y))

(3520, 128, 128)
[0. 1. 2. 3.]
(3520, 128, 128, 4)
[0. 1.]


In [ ]:
src="/content/input/train"
imag="/images/"
Siz=0
id=0
ri=0
metadat=[]
c="001"
for i,x in enumerate(sorted(glob.glob(src+imag+"*.png"))):
  if x.find("radiopaedia") == -1:
    x=x[x.find("_")+1:]
    x=x[:x.find("_")]
  else:
    x=x[x.find("_")+1:]
    x=x[:x.find("_",9)]
  if c!=x:
    print(x)
    metadat.append([id,Siz,ri])
    ri=i
    c=x
    id+=1
    Siz=0
  Siz+=1
metadat.append([id,Siz,ri])

002
003
004
005
006
007
008
009
010
10_85902_1
10_85902_3
14_85914_0
27_86410_0
29_86490_1
29_86491_1
36_86526_0
40_86625_0
4_85506_1
7_85703_0


##New Split

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
IDX=[x for x in range(10)]
IDY=[x for x in range(10)]
for train_index, val_index in cv.split(metadat,metadat):
  print(train_index, val_index)

[ 0  1  2  4  5  7  8  9 11 12 13 14 15 17 18 19] [ 3  6 10 16]
[ 0  1  3  5  6  7  8  9 10 11 12 13 15 16 18 19] [ 2  4 14 17]
[ 2  3  4  5  6  8  9 10 11 12 14 15 16 17 18 19] [ 0  1  7 13]
[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 16 17] [ 9 15 18 19]
[ 0  1  2  3  4  6  7  9 10 13 14 15 16 17 18 19] [ 5  8 11 12]


In [ ]:
for i,x in enumerate(train_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_train=X[ri:ri+size]
    Y_train=Y[ri:ri+size]
  else:
    X_train=np.concatenate((X_train,X[ri:ri+size]),axis=0)
    Y_train=np.concatenate((Y_train,Y[ri:ri+size]),axis=0)
for i,x in enumerate(val_index):
  size=metadat[x][1]
  ri=metadat[x][2]
  if i == 0:
    X_test=X[ri:ri+size]
    Y_test=Y[ri:ri+size]
  else:
    X_test=np.concatenate((X_test,X[ri:ri+size]),axis=0)
    Y_test=np.concatenate((Y_test,Y[ri:ri+size]),axis=0)

##Borrar directorio /input en caso de error

In [ ]:
!rm -r /content/input

##Resnet

In [ ]:
from keras.layers import Layer
import keras.backend as K
import torch.nn as nn
import tensorflow as tf

In [ ]:
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [ ]:
def dice_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      #None,128,128,1
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      #None,128,128
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += dice_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def iou_coef_multilabel(y_true, y_pred, smooth=1):
    dice=0
    numLabels=y_true.shape[3]
    for index in range(numLabels):
      y_t=tf.expand_dims(y_true[:,:,:,index],axis=3)
      y_p=tf.expand_dims(y_pred[:,:,:,index],axis=3)
      dice += iou_coef(y_t, y_p)
    return dice/numLabels # taking average

In [ ]:
def conv_block(X,f,d=0.1,group=1):
  c = tf.keras.layers.Conv2D(f[0], (3, 3), activation='relu', kernel_initializer='he_normal', padding='same',groups=group)(X)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  c = tf.keras.layers.Dropout(d)(c)
  c = tf.keras.layers.Conv2D(f[1], (3, 3), kernel_initializer='he_normal', padding='same', groups=group)(c)
  c = tf.keras.layers.BatchNormalization(axis=3)(c)
  s = tf.keras.layers.Conv2D(f[1], (1, 1), kernel_initializer='he_normal', padding='same')(X)
  s = tf.keras.layers.BatchNormalization(axis=3)(s)
  c = tf.keras.layers.Add()([c,s])
  c = tf.keras.layers.ReLU()(c)
  return c,s

In [ ]:
import tensorflow as tf
def create_model():
  g=4

  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
  #s= inputs
  #Contraction path
  c1,z1 = conv_block(s,[16,16],group=1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


  c2,z2 = conv_block(p1,[32,32],group=g)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3,z3 = conv_block(p2,[64,64],0.2,group=g)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c4,z4 = conv_block(p3,[128,128],0.2,group=g)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5,_ = conv_block(p4,[256,256],0.3,group=g)

  #Expansive path
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, z4])
  c6,_ = conv_block(u6,[128,128],0.2,group=g)

  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, z3])
  c7,_ = conv_block(u7,[64,64],0.2,group=g)

  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, z2])
  c8,_ = conv_block(u8,[32,32],group=g)

  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, z1], axis=3)
  c9,_ = conv_block(u9,[16,16],group=g)

  outputs = tf.keras.layers.Conv2D(4, (1, 1), activation='softmax')(c9)

  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',iou_coef_multilabel,dice_coef_multilabel])
  return model

##Model Fit

In [ ]:
model=create_model()
model.fit(X_train,Y_train,batch_size=16,epochs=25)

Epoch 1/25
158/158 [==============================] - 52s 116ms/step - loss: 0.3910 - accuracy: 0.8957 - iou_coef_multilabel: 0.2893 - dice_coef_multilabel: 0.3588
Epoch 2/25
158/158 [==============================] - 11s 71ms/step - loss: 0.0623 - accuracy: 0.9869 - iou_coef_multilabel: 0.5171 - dice_coef_multilabel: 0.5796
Epoch 3/25
158/158 [==============================] - 11s 69ms/step - loss: 0.0365 - accuracy: 0.9908 - iou_coef_multilabel: 0.5901 - dice_coef_multilabel: 0.6460
Epoch 4/25
158/158 [==============================] - 11s 69ms/step - loss: 0.0281 - accuracy: 0.9921 - iou_coef_multilabel: 0.6248 - dice_coef_multilabel: 0.6764
Epoch 5/25
158/158 [==============================] - 11s 70ms/step - loss: 0.0243 - accuracy: 0.9927 - iou_coef_multilabel: 0.6444 - dice_coef_multilabel: 0.6932
Epoch 6/25
158/158 [==============================] - 11s 71ms/step - loss: 0.0220 - accuracy: 0.9931 - iou_coef_multilabel: 0.6582 - dice_coef_multilabel: 0.7051
Epoch 7/25
158/158 [=

In [ ]:
scores= model.evaluate(X_test, Y_test, verbose=0)
print(f'Score for fold {0}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')

Score for fold 0: loss of 0.02801677957177162; accuracy of 99.20875430107117% iou_coef_multilabel of 70.94874978065491% dice_coef_multilabel of 74.46155548095703%


## Model Fit Kfold

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

25 epochs

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(X,Y):
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',DiceMetric])

  X_fold=X[train_index,:,:]
  Y_fold=Y[train_index,:,:]

  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,validation_split=0.2,verbose=0)

  Xtest_fold=X[val_index,:,:]
  Ytest_fold=Y[val_index,:,:]
  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}%')
  nfold+=1

Score for fold 1: loss of 0.010180109180510044; accuracy of 98.96095395088196% DiceMetric of 90.05458354949951%
Score for fold 2: loss of 0.02093512751162052; accuracy of 98.82934093475342% DiceMetric of 87.66295909881592%
Score for fold 3: loss of 0.021466268226504326; accuracy of 98.8848626613617% DiceMetric of 84.38072204589844%
Score for fold 4: loss of 0.02234414406120777; accuracy of 98.87327551841736% DiceMetric of 86.7185652256012%
Score for fold 5: loss of 0.020873798057436943; accuracy of 98.92811179161072% DiceMetric of 85.13778448104858%


50 epochs

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(X,Y):
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',DiceMetric])

  X_fold=X[train_index,:,:]
  Y_fold=Y[train_index,:,:]

  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,validation_split=0.2,verbose=0)

  Xtest_fold=X[val_index,:,:]
  Ytest_fold=Y[val_index,:,:]
  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}%')
  nfold+=1

Score for fold 1: loss of 0.026294833049178123; accuracy of 98.93321394920349% DiceMetric of 88.32351565361023%
Score for fold 2: loss of 0.03351094573736191; accuracy of 98.8107681274414% DiceMetric of 87.70761489868164%
Score for fold 3: loss of 0.03587321937084198; accuracy of 98.87182712554932% DiceMetric of 84.11082625389099%
Score for fold 4: loss of 0.03035399504005909; accuracy of 98.86696338653564% DiceMetric of 87.92186379432678%
Score for fold 5: loss of 0.02977365255355835; accuracy of 98.92091155052185% DiceMetric of 85.00602841377258%


##Actualized kfold

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.015655724331736565; accuracy of 99.492347240448% iou_coef_multilabel of 78.48652005195618% dice_coef_multilabel of 81.29676580429077%
Score for fold 2: loss of 0.033560991287231445; accuracy of 99.04505610466003% iou_coef_multilabel of 76.57327651977539% dice_coef_multilabel of 80.87905645370483%
Score for fold 3: loss of 0.027293214574456215; accuracy of 99.12284016609192% iou_coef_multilabel of 75.1055359840393% dice_coef_multilabel of 79.40298914909363%
Score for fold 4: loss of 0.02852698229253292; accuracy of 98.97468090057373% iou_coef_multilabel of 74.88389611244202% dice_coef_multilabel of 80.18468618392944%
Score for fold 5: loss of 0.025266241282224655; accuracy of 99.28717613220215% iou_coef_multilabel of 74.9929428100586% dice_coef_multilabel of 78.33638191223145%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)


  model.fit(X_fold,Y_fold,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.01602116785943508; accuracy of 99.52024221420288% iou_coef_multilabel of 82.14603662490845% dice_coef_multilabel of 84.78365540504456%
Score for fold 2: loss of 0.04220713675022125; accuracy of 98.94644618034363% iou_coef_multilabel of 81.23083114624023% dice_coef_multilabel of 85.52900552749634%
Score for fold 3: loss of 0.029687974601984024; accuracy of 99.19074773788452% iou_coef_multilabel of 78.43326330184937% dice_coef_multilabel of 82.49987959861755%
Score for fold 4: loss of 0.03447713330388069; accuracy of 98.83654117584229% iou_coef_multilabel of 79.59023714065552% dice_coef_multilabel of 85.02606749534607%
Score for fold 5: loss of 0.030355304479599; accuracy of 99.24253225326538% iou_coef_multilabel of 74.45892691612244% dice_coef_multilabel of 77.79458165168762%


##Model data augmented kfold Fixed

In [ ]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("vertical"),
  tf.keras.layers.RandomRotation(0.1),
])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

In [ ]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  X__2 = np.concatenate([X_fold, Y_fold], axis=-1)
  X__2 = tf.data.Dataset.from_tensor_slices((X__2))
  augmented_train_data = X__2.map(lambda x: data_augmentation(x, training=True))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x: x)
  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = X_train_augmented[:,:,:,1:5]
  X_train_augmented = X_train_augmented[:,:,:,0]

  X_train_combined = np.concatenate([X_fold, X_train_augmented[:,:,:,None]], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=50,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.014148196205496788; accuracy of 99.54409599304199% iou_coef_multilabel of 83.20818543434143% dice_coef_multilabel of 85.87126731872559%
Score for fold 2: loss of 0.040437109768390656; accuracy of 99.01766777038574% iou_coef_multilabel of 81.3275158405304% dice_coef_multilabel of 85.5164885520935%
Score for fold 3: loss of 0.030514268204569817; accuracy of 99.27120804786682% iou_coef_multilabel of 81.86497092247009% dice_coef_multilabel of 85.83415746688843%
Score for fold 4: loss of 0.024612445384263992; accuracy of 99.15140867233276% iou_coef_multilabel of 80.59779405593872% dice_coef_multilabel of 85.63371300697327%
Score for fold 5: loss of 0.025231849402189255; accuracy of 99.32593703269958% iou_coef_multilabel of 80.74448108673096% dice_coef_multilabel of 83.97288918495178%


In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
nfold=1
for train_index, val_index in cv.split(metadat,metadat):
  model = create_model()
  for i,x in enumerate(train_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      X_fold=X[ri:ri+size]
      Y_fold=Y[ri:ri+size]
    else:
      X_fold=np.concatenate((X_fold,X[ri:ri+size]),axis=0)
      Y_fold=np.concatenate((Y_fold,Y[ri:ri+size]),axis=0)
  for i,x in enumerate(val_index):
    size=metadat[x][1]
    ri=metadat[x][2]
    if i == 0:
      Xtest_fold=X[ri:ri+size]
      Ytest_fold=Y[ri:ri+size]
    else:
      Xtest_fold=np.concatenate((Xtest_fold,X[ri:ri+size]),axis=0)
      Ytest_fold=np.concatenate((Ytest_fold,Y[ri:ri+size]),axis=0)
  X__2 = np.concatenate([X_fold, Y_fold], axis=-1)
  X__2 = tf.data.Dataset.from_tensor_slices((X__2))
  augmented_train_data = X__2.map(lambda x: data_augmentation(x, training=True))

  # Separate X_train and Y_train from the augmented train data
  X_train_augmented = augmented_train_data.map(lambda x: x)
  X_train_augmented = np.asarray(list(X_train_augmented.as_numpy_iterator()))
  Y_train_augmented = X_train_augmented[:,:,:,1:5]
  X_train_augmented = X_train_augmented[:,:,:,0]

  X_train_combined = np.concatenate([X_fold, X_train_augmented[:,:,:,None]], axis=0)
  Y_train_combined = np.concatenate([Y_fold, Y_train_augmented], axis=0)
  model.fit(X_train_combined,Y_train_combined,batch_size=16,epochs=25,verbose=0)

  scores= model.evaluate(Xtest_fold, Ytest_fold, verbose=0)
  print(f'Score for fold {nfold}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}% {model.metrics_names[2]} of {scores[2]*100}% {model.metrics_names[3]} of {scores[3]*100}%')
  nfold+=1

Score for fold 1: loss of 0.013703295961022377; accuracy of 99.53930974006653% iou_coef_multilabel of 82.55035281181335% dice_coef_multilabel of 85.26062369346619%
Score for fold 2: loss of 0.04075786843895912; accuracy of 98.90410304069519% iou_coef_multilabel of 77.47979760169983% dice_coef_multilabel of 81.85679316520691%
Score for fold 3: loss of 0.029439277946949005; accuracy of 99.13827180862427% iou_coef_multilabel of 79.27287817001343% dice_coef_multilabel of 83.47063660621643%
Score for fold 4: loss of 0.024929743260145187; accuracy of 99.10342693328857% iou_coef_multilabel of 80.03262281417847% dice_coef_multilabel of 85.09231805801392%
Score for fold 5: loss of 0.022195545956492424; accuracy of 99.34304356575012% iou_coef_multilabel of 76.9676148891449% dice_coef_multilabel of 80.50482273101807%
